Торговый регистр Словакии располагается по ссылке: http://www.orsr.sk/search_osoba.asp?lan=en . 

Профиль каждой компании имеет ссылку вида: http://www.orsr.sk/vypis.asp?lan=en&ID=154017&SID=2&P=0

Задание: Скачайте 50000 профилей любых словацких компаний с этого сайта и вытащите из них всю информацию о людях (поля management, member, partners). В отдельный файл поместите тех, у кого в адресе упоминается Российская Федерация. Файл со всеми людьми и файл с людьми из России поместите в 2 csv файла формата название_компании;имя_человека;адрес_человека 

In [295]:
import pandas as pd
import os
from bs4 import BeautifulSoup as BS
import urllib.request 
import re

In [296]:
# создаем папку, куда качать профили компаний
folder_name = 'downloads'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [303]:
# функция для разбора страницы команиии
def process_soup(soup):
    df = pd.DataFrame(columns = ['company_name', 'name', 'address'])
    name_str = "";
    address_str = "";
    company_name = "-"
    # достаем имя компании　(если нет - кладём туда ICO)
    if(bool(soup.find(string=re.compile("Business name")))):
        company_name = soup.find(string=re.compile("Business name")).find_next("span").contents[0].strip() ######### company_name
    elif(bool(soup.find(string=re.compile("Identification number")))):
        company_name = soup.find(string=re.compile("Identification number")).find_next("span").contents[0].strip()
    # достаем менеджмент. имя - адрес
    if(bool(soup.find(string=re.compile("Management body:")))):
        managers = soup.find(string=re.compile("Management body:")).find_next("td").find_all("table")
        for manager in managers:
            # ссылка с именем 
            name_link = manager.find("a")

            # поиск имени
            if(name_link):
                name_str = "";
                name_items = name_link.find_all('span')
                for name_item in name_items:
                    for i in range(len(name_item)):
                        name_str += name_item.contents[i].strip() + " "
                name_str = name_str.strip()                                          ########## name

            # поиск адреса
            address = manager.find('br').find_next_siblings('span')
            if(address):
                address_str = "";
                for item in address:
                    address_str += item.contents[0].strip() + " "
                if('From:' in address_str):
                    address_str = address_str[:-len(' From: 06/21/2013')]            ########### address

            # создаем строку, добавляем во временную таблицу
            if(name_str and address_str):
                row = pd.Series([company_name, name_str, address_str], index = ['company_name', 'name', 'address'])
                df = df.append(row, ignore_index = True)

    # достаем партнеров. 
    if(bool(soup.find(string=re.compile("Partners:")))):
        partners = soup.find(string=re.compile("Partners:")).find_next("td").find_all("table")
        for partner in partners:
            # ссылка с именем 
            name_link = partner.find("a")

            # поиск имени
            if(name_link):
                name_str = "";
                name_items = name_link.find_all('span')
                for name_item in name_items:
                    for i in range(len(name_item)):
                        name_str += name_item.contents[i].strip() + " "
                name_str = name_str.strip()                                        ########## name

            # поиск адреса
            address = partner.find('br').find_next_siblings('span')
            if(address):
                address_str = "";
                for item in address:
                    address_str += item.contents[0].strip() + " "
                if('From:' in address_str):
                    address_str = address_str[:-len(' From: 06/21/2013')]           ########### address

            # создаем строку, добавляем во временную таблицу
            if(name_str and address_str):
                row = pd.Series([company_name, name_str, address_str], index = ['company_name', 'name', 'address'])
                df = df.append(row, ignore_index = True)

    # достаем супервайзеров. 
    if(bool(soup.find(string=re.compile("Supervisory board:")))):
        supervisors = soup.find(string=re.compile("Supervisory board:")).find_next("td").find_all("table")
        for supervisor in supervisors:

            # поиск имени
            name = str(supervisor).split('br')[0]
            sp_name = BS(name, "html.parser")
            name_str = "";
            name_items = sp_name.find_all('span')
            for name_item in name_items:
                for i in range(len(name_item)):
                    name_str += name_item.contents[i].strip() + " "
            name_str = name_str.strip() 

            # удаляем ненужные приставки к имени
            name_len = len(name_str.split(" "))
            if(name_len > 2):
                name_split = name_str.split(" ")[1:]
                name_str = " ".join(name_split)                                   ########## name

            # поиск адреса
            address1 = str(supervisor).split('br')[1]
            if(('from' not in address1) and (len(str(supervisor).split('br')) > 2)):
                address2 = str(supervisor).split('br')[2]

                address = address1 + address2
                sp_address = BS(address, "html.parser")
                address_str = "";
                address_items = sp_address.find_all('span')
                for address_item in address_items:
                    for i in range(len(address_item)):
                        address_str += address_item.contents[i].strip() + " "
                address_str = address_str.strip()                                 ########### address

            # создаем строку, добавляем во временную таблицу
            if(name_str and address_str):
                row = pd.Series([company_name, name_str, address_str], index = ['company_name', 'name', 'address'])
                df = df.append(row, ignore_index = True)
    # убираем повторяющихся людей 
    df = df.drop_duplicates()
    return df


In [302]:
df_all.shape

(44188, 3)

In [304]:
from IPython.display import clear_output
# запрашиваем информацию по 50 000 компаний (если найдем столько)
# берем только те компании, у которых есть информация о людях

#df_all = pd.DataFrame(columns = ['company_name', 'name', 'address'])
company_id = 34883
#saved_count = 0
while saved_count < 50000:
    url = "http://www.orsr.sk/vypis.asp?lan=en&ID=" + str(company_id) + "&SID=2&P=0"
    file_path = os.path.join(folder_name, str(company_id)+".html")
    response = urllib.request.urlopen(url)
    html_response = response.read()
    encoding = response.headers.get_content_charset('windows-1250')
    decoded_html = html_response.decode(encoding)
    
    ########## Этот кусок для сохранения файлов на диск######################
    #with open(file_path,"w", encoding="windows-1250") as file:
    #    file.write(decoded_html)
    #    file.close()
    #with open (file_path, 'r', encoding='windows-1250') as file:
    #    soup = BS(file, 'lxml', from_encoding="windows-1250")
    #########################################################################
    
    #### обработка данных без сохранения файлов
    soup = BS(decoded_html, 'lxml', from_encoding="windows-1250")
    
    # не рассматриваем запись, если это самозанятый гражданин
    is_self_employed = soup.find(string=re.compile("Self-employed individual"))
    has_management = bool(soup.find(string=re.compile("Management body")))
    has_member = bool(soup.find(string=re.compile("Contribution of each member")))
    has_partners = bool(soup.find(string=re.compile("Partners")))
    
    # если всё устраивает, извлекаем данные.
    if((not is_self_employed) and (has_management or has_member or has_partners)):
        df = process_soup(soup)
        df_all = df_all.append(df, ignore_index=True)
        saved_count += 1
    company_id += 1
    
    clear_output(wait = True)
    print('Processed % 2d companies out of % 2d ids' %(saved_count, company_id))
df_all.to_csv('slovakia_companies.csv', index=False)
df_all.tail()

Processed  50000 companies out of  177632 ids


,company_name,name,address
108662,CUSTOM SOLUTIONS s.r.o.,Alan Glozik,Jána Smreka 14 Bratislava-Devínska Nová Ves 84...
108663,CUSTOM SOLUTIONS s.r.o.,Helena Gloziková,Lachova 15 Bratislava-Petržalka 851 03
108664,CUSTOM SOLUTIONS s.r.o.,Alan Glozik,Jána Smreka 14 Bratislava 841 08
108665,CUSTOM SOLUTIONS s.r.o.,Helena Gloziková,Lachova 15 Bratislava 851 03
108666,"OVO PACK, s. r. o.",Gabriel Kuczman,Tatranská 172/1016 Nové Zámky 940 01


дошли до ID = 177632

In [305]:
df_50k = pd.read_csv('slovakia_companies.csv')
df_50k.head()

,company_name,name,address
0,Poľnohospodárske družstvo Vajnory,Branislav Brúder,Ľublanská 2 Bratislava 831 02
1,Poľnohospodárske družstvo Vajnory,Renáta Sláviková,Pristruhe 44 Bratislava 831 07
2,Poľnohospodárske družstvo Vajnory,Peter Ondris,Dorastenecká 4 Bratislava 831 07
3,Poľnohospodárske družstvo Vajnory,Anna Feketová,Roľnícka 57 Bratislava 831 07
4,Poľnohospodárske družstvo Vajnory,Janka Šmidová,Roľnícka 262 Bratislava 831 07


In [292]:
# Для теста отдельных файлов!
company_id = 4012
url = "http://www.orsr.sk/vypis.asp?lan=en&ID=" + str(company_id) + "&SID=2&P=0"
file_path = os.path.join(folder_name, str(company_id)+".html")
response = urllib.request.urlopen(url)
html_response = response.read()
encoding = response.headers.get_content_charset('windows-1250')
decoded_html = html_response.decode(encoding)
soup = BS(decoded_html, 'lxml', from_encoding="windows-1250")
########## Этот кусок для сохранения файлов на диск######################
#with open(file_path,"w", encoding="windows-1250") as file:
#    file.write(decoded_html)
#    file.close()
#with open (file_path, 'r', encoding='windows-1250') as file:
#    soup = BS(file, 'lxml', from_encoding="windows-1250")

#with open ("5.html", 'r', encoding='windows-1250') as file:
#    soup = BS(file, 'lxml', from_encoding="windows-1250")
    
df = pd.DataFrame(columns = ['company_name', 'name', 'address'])
name_str = "";
address_str = "";
# достаем имя компании
company_name = soup.find(string=re.compile("Business name:")).find_next("span").contents[0].strip() ######### company_name

# достаем менеджмент. имя - адрес
if(bool(soup.find(string=re.compile("Management body:")))):
    managers = soup.find(string=re.compile("Management body:")).find_next("td").find_all("table")
    for manager in managers:
        # ссылка с именем 
        name_link = manager.find("a")

        # поиск имени
        if(name_link):
            name_str = "";
            name_items = name_link.find_all('span')
            for name_item in name_items:
                for i in range(len(name_item)):
                    name_str += name_item.contents[i].strip() + " "
            name_str = name_str.strip()                                                                ########## name

        # поиск адреса
        address = manager.find('br').find_next_siblings('span')
        if(address):
            address_str = "";
            for item in address:
                address_str += item.contents[0].strip() + " "
            if('From:' in address_str):
                address_str = address_str[:-len(' From: 06/21/2013')]                                       ########### address

        # создаем строку, добавляем во временную таблицу
        if(name_str and address_str):
            row = pd.Series([company_name, name_str, address_str], index = ['company_name', 'name', 'address'])
            df = df.append(row, ignore_index = True)

# достаем партнеров. 
if(bool(soup.find(string=re.compile("Partners:")))):
    partners = soup.find(string=re.compile("Partners:")).find_next("td").find_all("table")
    for partner in partners:
        # ссылка с именем 
        name_link = partner.find("a")

        # поиск имени
        if(name_link):
            name_str = "";
            name_items = name_link.find_all('span')
            for name_item in name_items:
                for i in range(len(name_item)):
                    name_str += name_item.contents[i].strip() + " "
            name_str = name_str.strip()                                                                ########## name

        # поиск адреса
        address = partner.find('br').find_next_siblings('span')
        if(address):
            address_str = "";
            for item in address:
                address_str += item.contents[0].strip() + " "
            if('From:' in address_str):
                address_str = address_str[:-len(' From: 06/21/2013')]                               ########### address

        # создаем строку, добавляем во временную таблицу
        if(name_str and address_str):
            row = pd.Series([company_name, name_str, address_str], index = ['company_name', 'name', 'address'])
            df = df.append(row, ignore_index = True)
            
# достаем супервайзеров. 
if(bool(soup.find(string=re.compile("Supervisory board:")))):
    supervisors = soup.find(string=re.compile("Supervisory board:")).find_next("td").find_all("table")
    for supervisor in supervisors:
        
        # поиск имени
        name = str(supervisor).split('br')[0]
        sp_name = BS(name, "html.parser")
        name_str = "";
        name_items = sp_name.find_all('span')
        for name_item in name_items:
            for i in range(len(name_item)):
                name_str += name_item.contents[i].strip() + " "
        name_str = name_str.strip() 
        
        # удаляем ненужные приставки к имени
        name_len = len(name_str.split(" "))
        if(name_len > 2):
            name_split = name_str.split(" ")[1:]
            name_str = " ".join(name_split)     
        # поиск адреса
        address1 = str(supervisor).split('br')[1]
        if(('from' not in address1) and (len(str(supervisor).split('br')) > 2)):
            address2 = str(supervisor).split('br')[2]

            address = address1 + address2
            sp_address = BS(address, "html.parser")
            address_str = "";
            address_items = sp_address.find_all('span')
            for address_item in address_items:
                for i in range(len(address_item)):
                    address_str += address_item.contents[i].strip() + " "
            address_str = address_str.strip()                                                        ########### address

        # создаем строку, добавляем во временную таблицу
        if(name_str and address_str):
            row = pd.Series([company_name, name_str, address_str], index = ['company_name', 'name', 'address'])
            df = df.append(row, ignore_index = True)
df = df.drop_duplicates()
df

2
2
2


C:\Users\lobov\Anaconda3\envs\py3\lib\site-packages\bs4\__init__.py:177: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


,company_name,name,address
0,"Detský kupónový fond, investičný fond v skráte...",Ivica Adamcová,367/3 Vrbové
1,"Detský kupónový fond, investičný fond v skráte...",Eliška Dančová,Revová 21 Bratislava
2,"Detský kupónový fond, investičný fond v skráte...",Radovan Ujházy,91 Rožňavské Bystré
3,"Detský kupónový fond, investičný fond v skráte...",Vladimír Bilčík,91 Rožňavské Bystré
4,"Detský kupónový fond, investičný fond v skráte...",Vladimír Ďurovčík - člen,91 Rožňavské Bystré
5,"Detský kupónový fond, investičný fond v skráte...",Peter Chrappa - člen,91 Rožňavské Bystré


## TODO: 
- Получить компании с русскими именами и явками
- Сохранить это в отдельный файл